In [ ]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('Folium installed')
print('Libraries imported.')

Solving environment: - 

In [1]:
import pandas as pd
data = pd.read_csv('http://cocl.us/Geospatial_data')
data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [56]:
data = pd.DataFrame(data)
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686299999996,-79.19435340000001
1,M1C,43.7845351,-79.16049709999999
2,M1E,43.7635726,-79.1887115
3,M1G,43.7709921,-79.21691740000001
4,M1H,43.773136,-79.23947609999999


In [3]:
import urllib.request
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")
all_tables=soup.findAll("table")
right_table = soup.find('table',{'class':'wikitable'})
A=[]
B=[]
C=[]
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
import pandas as pd
df=pd.DataFrame(A,columns=['Postcode'])
df['Borough']=B
df['Neighborhood']=C
df.head()


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [4]:
df1=df.drop([0,1,7,10,15,16,19,24,25,28,29,33,34,35,37,38,42,43,44,51,52,53,60,61,62,69,70,71,78,79,87,88,96,97,101,105,106,110,115,118,119,123,124,125,127,128,131,132,133,134,136,137,140,141,145,146,149,150,154,155,158,159,161,162,163,164,166,167,170,171,172,173,174,175,176,177,179])
df1.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [5]:
df1.reset_index(drop=True, inplace=True)
df1.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [17]:
df2=df1.sort_values(by=['Postcode'])
df2.head()
df2.reset_index(drop=True, inplace=True)
df2.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [48]:
df3 =df2.rename(columns ={'Postcode':'Postal Code'})
df3.head()


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [57]:
data.reset_index(drop=True, inplace=True)

df3.reset_index(drop=True, inplace=True)
Result = pd.concat([df3, data], axis=1)
Result

,Postal Code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,M1B,43.806686299999996,-79.19435340000001
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,M1C,43.7845351,-79.16049709999999
2,M1E,Scarborough,Guildwood / Morningside / West Hill,M1E,43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,M1G,43.7709921,-79.21691740000001
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.23947609999999
5,M1J,Scarborough,Scarborough Village,M1J,43.7447342,-79.23947609999999
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,M1K,43.7279292,-79.26202940000002
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,M1L,43.711111700000004,-79.2845772
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village W...,M1M,43.716316,-79.23947609999999
9,M1N,Scarborough,Birch Cliff / Cliffside West,M1N,43.692657000000004,-79.2648481


In [58]:
My_data =  pd.merge(df3, data, on='Postal Code', how='outer')

In [59]:
My_data = pd.merge(df3,data[['Postal Code', 'Latitude', 'Longitude']],
                 on='Postal Code', 
                 how='outer')